<a href="https://colab.research.google.com/github/j0gea/Machine-Learning-Sidewalk-damage/blob/main/yolov5_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 구글 드라이브 파일 압축풀기

- s 는 추가파일이라 할 필요 x

In [2]:
%cd /content/drive/MyDrive/yolov5/DATA

/content/drive/MyDrive/yolov5/DATA


In [3]:
!ls

 data   label_temp  '베리어프리존(장애물 없는 생활공간) 주행영상'


In [4]:
# 라벨 파일 압축풀기
%cd /content/drive/MyDrive/yolov5/DATA/label_temp

!unzip -qq "/content/drive/MyDrive/yolov5/DATA/베리어프리존(장애물 없는 생활공간) 주행영상/[라벨]2.서부산실외_베리어프리객체_1.zip"

/content/drive/MyDrive/yolov5/DATA/label_temp


In [5]:
# 그림 파일 압축풀기
%cd /content/drive/MyDrive/yolov5/DATA/data/images

!unzip -qq "/content/drive/MyDrive/yolov5/DATA/베리어프리존(장애물 없는 생활공간) 주행영상/[원천]2.서부산실외_베리어프리객체_1.zip"

/content/drive/MyDrive/yolov5/DATA/data/images


In [ ]:
# 파일 삭제
# !rm -rf ./*.txt
# !rm -rf ./*.json

### YOLOv5 설치

yolov5 설정

In [ ]:
%cd 설치 폴더로 이동

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 16802, done.
remote: Counting objects: 100% (343/343), done.
remote: Compressing objects: 100% (249/249), done.
remote: Total 16802 (delta 186), reused 179 (delta 94), pack-reused 16459
Receiving objects: 100% (16802/16802), 15.56 MiB | 12.62 MiB/s, done.
Resolving deltas: 100% (11485/11485), done.


In [ ]:
%cd yolov5

In [ ]:
! pip install -r requirements.txt

## YOLOv5 Colab
- https://velog.io/@jiiina/YOLO-Colab
- https://clap0107.tistory.com/5
- 저거써볼거임

- 생각해보니 JSON 파일도 함께 사용해야함
- https://velog.io/@1-june/YOLOv5%EC%97%90%EA%B2%8C-%EC%83%88%EB%A1%9C%EC%9A%B4-%EA%B2%83-%EA%B0%80%EB%A5%B4%EC%B3%90%EB%B3%B4%EA%B8%B0
- https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#local-logging

In [ ]:
# 런타임 > 런타임 유형 변경> 하드웨어 가속기에서 GPU를 선택
# GPU 제대로 잡히는지 확인

### YOLO 실행

#### 라벨 to txt

txt 변환
- 참고
- https://velog.io/@1-june/YOLOv5%EC%97%90%EA%B2%8C-%EC%83%88%EB%A1%9C%EC%9A%B4-%EA%B2%83-%EA%B0%80%EB%A5%B4%EC%B3%90%EB%B3%B4%EA%B8%B0
- https://minmiin.tistory.com/14

In [ ]:
# 어차피 여기서 다시입히기때문에 사진 크기 작게 하고 비율로 해도 될것같음

In [14]:
import re
import json
import os
import csv

training_labels_folder = "/content/drive/MyDrive/yolov5/DATA/label_temp"
training_images_folder = "/content/drive/MyDrive/yolov5/Data/testdata/images"
jsons = []

#for f in os.listdir(training_images_folder):
#  jsons.append(f[0:-3]+"json")

for f in os.listdir(training_labels_folder):
  jsons.append(f)

len(jsons)

5538

In [35]:
def save_as_yolo_format(destination_folder, json_data):
    img_width = json_data['images']['width'] # JSON 파일에서 이미지의 width 값이 있는 곳
    img_height = json_data['images']['height'] # JSON 파일에서 이미지의 height 값이 있는 곳

    yolov5_format_list = []
    for i in json_data['annotations']: # 고려할 것들
        if i['category_id'] == 'flatness_D' or i['category_id'] == 'flatness_E' or i['category_id'] == 'block_state_broken':
          region_num = 0
        else:
          region_num = 1
        xmax = -1; xmin = 5000; ymax = -1; ymin = 5000
        for x, y in (i['segmentation']):
            if (x > xmax): xmax = x
            if (x < xmin): xmin = x
            if (y > ymax): ymax = y
            if (y < ymin): ymin = y

        xcentre = ((xmax + xmin) / 2) / json_data['images']['width']
        ycentre = ((ymax + ymin) / 2) / json_data['images']['height']
        bbox_width = (xmax - xmin) / json_data['images']['width']
        bbox_height = (ymax - ymin) / json_data['images']['height']
        yolov5_format = [region_num, xcentre, ycentre, bbox_width, bbox_height]
        yolov5_format_list.append(yolov5_format)

    file = open(os.path.join(destination_folder, re.sub(r'[^.]+$', 'txt', json_data['images']['file_name'])), 'w', newline='') # 파일명
    with file:
        write = csv.writer(file, delimiter= ' ')
        write.writerows(yolov5_format_list)

In [38]:
destination_folder = "/content/drive/MyDrive/yolov5/DATA/data/labels"

n=1
for j in jsons:
  with open(os.path.join(training_labels_folder, j)) as json_file:
    json_data = json.load(json_file)
    save_as_yolo_format(destination_folder, json_data)
    print(n,"/",len(jsons))
    n=n+1

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
539 / 5538
540 / 5538
541 / 5538
542 / 5538
543 / 5538
544 / 5538
545 / 5538
546 / 5538
547 / 5538
548 / 5538
549 / 5538
550 / 5538
551 / 5538
552 / 5538
553 / 5538
554 / 5538
555 / 5538
556 / 5538
557 / 5538
558 / 5538
559 / 5538
560 / 5538
561 / 5538
562 / 5538
563 / 5538
564 / 5538
565 / 5538
566 / 5538
567 / 5538
568 / 5538
569 / 5538
570 / 5538
571 / 5538
572 / 5538
573 / 5538
574 / 5538
575 / 5538
576 / 5538
577 / 5538
578 / 5538
579 / 5538
580 / 5538
581 / 5538
582 / 5538
583 / 5538
584 / 5538
585 / 5538
586 / 5538
587 / 5538
588 / 5538
589 / 5538
590 / 5538
591 / 5538
592 / 5538
593 / 5538
594 / 5538
595 / 5538
596 / 5538
597 / 5538
598 / 5538
599 / 5538
600 / 5538
601 / 5538
602 / 5538
603 / 5538
604 / 5538
605 / 5538
606 / 5538
607 / 5538
608 / 5538
609 / 5538
610 / 5538
611 / 5538
612 / 5538
613 / 5538
614 / 5538
615 / 5538
616 / 5538
617 / 5538
618 / 5538
619 / 5538
620 / 5538
621 / 5538
622 / 5538
623 / 5538
624 / 5538
625 / 5538
626 / 5

In [ ]:
# 테스트코드

import json
import csv
import os
from PIL import Image, ImageDraw
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

training_images_folder = "/content/drive/MyDrive/yolov5/DATA/data/images"

img_path = "/content/drive/MyDrive/yolov5/DATA/data/images/DC0106_2020-11-10 135055_00.jpg"
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

draw_img = Image.fromarray(img)
draw = ImageDraw.Draw(draw_img)

img_width = img.shape[1]
img_height = img.shape[0]
print(img_width, img_height)

with open("/content/drive/MyDrive/yolov5/DATA/data/labels/DC0106_2020-11-10 135055_00.txt") as f:
    csv_reader = csv.reader(f, delimiter=' ')
    for row in csv_reader:
        xcentre = float(row[1]) * img_width
        ycentre = float(row[2]) * img_height
        bbox_width = float(row[3]) * img_width
        bbox_height = float(row[4]) * img_height
        x0 = xcentre - bbox_width/2
        x1 = xcentre + bbox_width/2
        y0 = ycentre - bbox_height/2
        y1 = ycentre + bbox_height/2

        draw.rectangle([x0, y0, x1, y1], outline='white', width=2)

plt.figure(figsize=(20, 10))
plt.imshow(draw_img)

#### train / val 나누기

In [10]:
# 먼저 export파일 안에 있는 이미지들을 list로 만들어준다.
from glob import glob
img_list = glob('/content/drive/MyDrive/yolov5/DATA/data/images/*.jpg')
print(len(img_list))

5538


In [11]:
# 다음에 img_list를 train set과 valid set으로 나눠준다.

from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size = 0.2, random_state = 2000)
print(len(train_img_list), len(val_img_list))

4430 1108


In [13]:
file = open("/content/drive/MyDrive/yolov5/DATA/train.txt", 'w', newline='') # 파일명
for i in train_img_list:
    file.write(i+"\n")
#for i in train_label_list:
#    file.write(i+"\n")
file.close()

In [14]:
file = open("/content/drive/MyDrive/yolov5/DATA/valid.txt", 'w', newline='') # 파일명
for i in val_img_list:
    file.write(i+"\n")
#for i in val_label_list:
#    file.write(i+"\n")
file.close()

#### yaml 파일 생성

In [18]:
%cd /content/drive/MyDrive/yolov5/DATA

/content/drive/MyDrive/yolov5/DATA


In [19]:
import yaml

yaml_data = {"names":['broken', 'not_broken'], # 클래스 이름
             "nc":2, # 클래스 수
             "path":"/content/drive/MyDrive/yolov5/DATA", # root 경로
             "train":("train.txt"), # train.txt 경로
             "val":("valid.txt"), # valid.txt 경로
             # "test":os.path.join(root_dir,"test.txt") # test.txt 경로
             }

with open(("custom.yaml"), "w") as f:
  yaml.dump(yaml_data, f)

#### YOLOv5 실행
- 각 파라메터 뜻 https://ropiens.tistory.com/44
- https://github.com/ultralytics/yolov5/blob/master/data/coco128.yaml
- https://sguys99.github.io/ds01

In [39]:
%cd /content/drive/MyDrive/yolov5/YOLO/yolov5

/content/drive/MyDrive/yolov5/YOLO/yolov5


In [ ]:
!python train.py --img 416 --batch 16 --epochs 50 --data /content/drive/MyDrive/yolov5/DATA/custom.yaml --cfg ./models/yolov5m.yaml --weights yolov5m.pt --name results

2024-07-23 05:12:48.855655: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 05:12:48.855706: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 05:12:48.857132: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5m.pt, cfg=./models/yolov5m.yaml, data=/content/drive/MyDrive/yolov5/DATA/custom.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=

In [ ]:
#  runs/train/results13